<a href="https://colab.research.google.com/github/KjelleJ/enkla-ai-experiment/blob/main/AX10_katt_eller_hund.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Katt eller hund?
Djupinlärning var tidigt framgångsrikt med klassificering av bilder. 2014 startade **Kaggle** en tävling där deltagarna skulle klassificera bilder av katter och hundar. Klasserna var katt och hund. Vinnarna använde djupinlärning som 2014 var ganska nytt. 95% noggrannhet var ett toppresultat. 25,000 bilder användes för träningen, 12,500 på katter och 12,500 på hundar.

Vi ska använda betydligt färre bilder. Att inte använda så många bilder är egentligen ett svårare men ett mera realistiskt problem eftersom det inte är lätt att få tag på tusentals relevanta bilder.

Vi kommer att amvända bilder på katter och hundar från Kaggle

*   För **träning**: Bilder på 250 katter och 250 hundar.
*   För **validering** Bilder på 125 katter och 125 hundar. Valideringsdata används för att utvärdea modellen under träningen. Modellen tränas inte med valideringsdata.
*   För **test** Bilder på 500 katter och 500 hundar.


Vi kommer att få bättre resultat än 95% noggrannhet men då använder vi metoder som deltagarna i tävlingen inte kunde använda.

**Du bör använda en GPU** - L4 duger bra: Pil uppe till höger, välj 'Ändra körningstyp', sedan välj L4 och spara.

In [ ]:
# Hämta bilderna - en zip-fil
!rm -f dogs_vs_cats_500.zip*
!wget download.gubboit.se/dogs_vs_cats_500.zip

In [ ]:
# Packa upp zip-filen
!unzip -oq dogs_vs_cats_500.zip

## Plotta de tre första hundarna och katterna i träningsdata

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

plt.figure(figsize=(12, 8))
for i in range(3):
    image = Image.open("dogs_vs_cats_500/train/cat/cat." + str(i) + ".jpg")
    plt.subplot(2, 3, i + 1)
    plt.axis('off')
    plt.imshow(image)
for i in range(0, 3):
    image = Image.open("dogs_vs_cats_500/train/dog/dog." + str(i) + ".jpg")
    plt.subplot(2, 3, i + 4)
    plt.axis('off')
    plt.imshow(image)

## Hjälpfunktioner

In [ ]:
# Funktion för att skapa datset för träning, validering och test
def create_datasets(new_base_dir, size):
    from tensorflow.keras.utils import image_dataset_from_directory

    print('Training')
    train = image_dataset_from_directory(
        new_base_dir + "/train",
        image_size=size,
        batch_size=32)
    print('Validation')
    validation = image_dataset_from_directory(
        new_base_dir + "/validation",
        image_size=size,
        batch_size=32)
    print('Test')
    test = image_dataset_from_directory(
        new_base_dir + "/test",
        image_size=(size),
        batch_size=32)
    return train, validation, test

In [ ]:
# funktion som plottar tränings-historiken
def plot_acc_loss():
    import matplotlib.pyplot as plt
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, "bo", label="Training accuracy")
    plt.plot(epochs, val_acc, "b", label="Validation accuracy")
    plt.title("Training and validation accuracy")
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, "bo", label="Training loss")
    plt.plot(epochs, val_loss, "b", label="Validation loss")
    plt.title("Training and validation loss")
    plt.legend()
    plt.show()

---
# Transfer learning: Modell för bilder med storleken 160x160 eller 224x224 pixlar
Att träna en modell för katt eller hund från scratch tar lång tid och kräver mera data än vi har för att få ett bra resultat. Vi ska i stället använda en smartare teknik **transfer learning** för att få bättre resultat. Transfer learning innebär att vi använder **en färdigtränad modell som bas.** Modellen har tränats med ett visst dataset. Vi ska använda MobileNetV2 som har tränats med datasetet Imagenet. Imagenet har 1000 klasser bl.a. mycket bilder på djur. Vi har bara två klasser (katt eller hund). Vi byter ut några lager i slutet av modellen (för 1000 klasser) så att vi får två klasser. Slutet kallas **toppen**. Vi tränar sedan den nya modellen men bara vikterna i den nya toppen. **Vikterna i basen är frysta**.

Vår modell ska använda **data augmentation** (”utökning av data”). Det är en teknik som används för att på konstlad väg öka antalet bilder vid träningen. På så vis kan vi minska risken för överanpassning av modellen till träningsdata och få ett bättre resultat. Data augmentation använder sig av slumpmässig horisontell flip (vänd bilden), rotation och zoomning. Åtminstone i teorin används inte samma bild två gånger vid träningen men bilderna är förstås ganska lika.

In [ ]:
# Kör först med npix=160 och sedn med npix=224
npix=160
#npix=224

In [ ]:
# skapa dataset från "dogs_vs_cats_500"
train_dataset, validation_dataset , test_dataset = create_datasets("dogs_vs_cats_500", (npix, npix))

In [ ]:
# Definiera basen: MobileNet utan topp och med alla vikter frysta
from tensorflow import keras

conv_base = keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(npix, npix, 3))
conv_base.trainable = False

In [ ]:
conv_base.summary()

In [ ]:
# Definiera den nya toppen. Här tränas alla vikter
from tensorflow import keras
from tensorflow.keras import layers

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)

inputs = keras.Input(shape=(npix, npix, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
# Vi sparar den bästa modellen från träningen på fil
if npix==160:
    filepath="feature_extraction2.keras"
else:
    filepath="feature_extraction3.keras"

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=filepath,
        save_best_only=True,
        monitor="val_loss")
]
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
# Utvärdera modellen
test_model = keras.models.load_model(filepath)
test_loss, test_acc = test_model.evaluate(test_dataset, verbose=0)
print(f"Test accuracy: {test_acc:.3f}  Test loss: {test_loss:.3f}")
if npix == 160:
  test_loss_160 = test_loss
  test_acc_160 = test_acc
else:
  test_loss_224 = test_loss
  test_acc_224 = test_acc

In [ ]:
# Plotta accuracy och loss från träningen
plot_acc_loss()

## Om du inte har kört med npix=224 gå till början av Transfer Learning och ändra npix=160 till npix=224 och kör med den nya bildstorleken. Annars fortsätt bara...

In [ ]:
# Jämför resultat för npix=160 och npix=224
print(f"Bildstorlek 160x160: Test accuracy: {test_acc_160:.3f}  Test loss: {test_loss_160:.3f}")
print(f"Bildstorlek 224x224: Test accuracy: {test_acc_224:.3f}  Test loss: {test_loss_224:.3f}")

# Använd modellen för bildstorlek 224x224
#### Bilden bör prepareras på samma sätt som vid träningen
Vi använde Keras-funktionen image_dataset_from_directory när dataseten skapades. Cat kommer före dog. Då bör cat vara klass 0 och dog klass 1.

In [ ]:
# modell för bilder 224x224
test_model = keras.models.load_model("feature_extraction3.keras")

In [ ]:
# funktion för att klassificera bilder
def catvsdog(img_path):
    import tensorflow as tf
    img = plt.imread(img_path)
    plt.imshow(img)
    # kod från https://keras.io/examples/vision/image_classification_from_scratch/
    img = tf.image.resize(img, (224, 224))
    img_array = keras.preprocessing.image.img_to_array(img)
    # add batch dim
    img_array = tf.expand_dims(img_array, 0)
    pred =  test_model.predict(img_array, verbose=0)
    # binary classification: tf-model returns 0 or 1
    if pred == 1: print("Hund")
    else: print("Katt")
    print(pred)


In [ ]:
# bild nr från 750 till och med 1249
import tensorflow as tf
catvsdog("dogs_vs_cats_500/test/cat/cat.752.jpg")


In [ ]:
# här blir det kanske fel!?
catvsdog("dogs_vs_cats_500/test/cat/cat.753.jpg")

In [ ]:
catvsdog("dogs_vs_cats_500/test/dog/dog.1249.jpg")

---
# MobileNet utan träning med cats_vs_dogs

### Imagenet klasser: https://gist.github.com/ageitgey/4e1342c10a71981d0b491e1b8227328b
### Katter 281-285, tamhundar 151-268, totalt 1000 klasser

MobileNet/Imagenet är bra på hundar och katter. Det finns 5 klasser för olika raser av katter och 118 klasser för olika hundraser. Vilken noggrannhet får vi om använder modellen som den är utan träning? Vi låter modellen göra prediktioner för våra testdata. Om prediktionen blir någon av kattklasserna säger vi att det är en katt. Motsvarande för hundklasserna. Om något annat så är det fel. Det hela fixas med lite Python-kod.

In [ ]:
# HELA MobileNetV2
mobile_net = keras.applications.MobileNetV2(
    weights="imagenet",
    include_top=True,
    input_shape=(160, 160, 3))

In [ ]:
mobile_net.summary()

In [ ]:
import numpy as np
import tensorflow as tf
hund = 0
for i in range(750, 1250):
    pimage = plt.imread("dogs_vs_cats_500/test/dog/dog." + str(i) + ".jpg")
    # lägg till batch- dimension, normalisera till (0.0, 1.0)
    pimage = pimage.astype(np.float32)[np.newaxis, ...] / 255.
    # ändra storlek till 160x160
    pimage = tf.image.resize(pimage, (160, 160))
    # bestäm klass
    pred = (mobile_net.predict(pimage, verbose=0)).argmax()
    if pred >= 151 and pred <= 268: hund = hund + 1
print(str(hund) + " hundar av 500 korrekt klassificerade. Noggrannhet=" + str(hund/500))

In [ ]:
katt = 0
for i in range(750, 1250):
    pimage = plt.imread("dogs_vs_cats_500/test/cat/cat." + str(i) + ".jpg")
    pimage = pimage.astype(np.float32)[np.newaxis, ...] / 255.
    pimage = tf.image.resize(pimage, (160, 160))
    pred = (mobile_net.predict(pimage, verbose=0)).argmax()
    if pred >= 281 and pred <= 285: katt = katt + 1
print(str(katt) + " katter av 500 korrekt klassificerade. Noggrannhet=" + str(katt/500))

In [ ]:
print("Noggrannhet totalt=" + str((hund + katt)/1000))

---
# Som jämförelse: Träna en enkel modell från scratch med lika mycket data
---

In [ ]:
 # definiera en enkel modell med 3 conv-lager
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
# sigmoid eftersom modellen har två klasser
outputs = layers.Dense(1, activation="sigmoid")(x)
model3 = keras.Model(inputs=inputs, outputs=outputs)

# binary_crossentropy eftersom modellen har 2 klasser
model3.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="model3_from_scratch.keras",
        save_best_only=True,
        monitor="val_loss")
]

model3.summary()

## Träna vår enkla modell från scratch i 100 epoker

In [ ]:
history = model3.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

## Utvärdera modellen

In [ ]:
# ladda och utvärdera bästa modellen (0.8250 med 2000 filer)
test_model = keras.models.load_model("model3_from_scratch.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"Test accuracy: {test_acc:.3f}")

In [ ]:
plot_acc_loss()

## Plottarna visar en tydlig överanpassning - kurvorna för träning och validering divergerar. För att få ett bättre resultat: Öka antalet träningsbilder och/eller använd en mera avancerad modell...